# Tennis Players Pose Detection

## Importing libraries

In [ ]:
import os
import io
import base64
from IPython.display import YouTubeVideo, HTML
from os.path import exists, join, basename, splitext

## Installing OpenPose

In [ ]:
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # installing new CMake because of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # cloning openpose repository
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # installing system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # installing python dependencies
  !pip install -q youtube-dl
  # building openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

## Downloading Video

In [ ]:
# saving YouTube video ID
YouTubeID = 'kHrAaIFOh2I'
# opening YouTube video
YouTubeVideo(YouTubeID)

In [ ]:
# deleting previous input video
!rm -rf youtube.mp4
# downloading the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YouTubeID
# cuting the first seconds of the video
!ffmpeg -y -loglevel info -i youtube.mp4 -t 16 video.mp4

## Detecting Poses

In [ ]:
# deleting previous output video
!rm openpose.avi
# detecting poses on the these first seconds
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# converting the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

## Showing Poses

In [ ]:
# reading the video
video_encoded = base64.b64encode(io.open('output.mp4', 'rb').read())
# displaying the video
HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
             <source src="data:video/mp4;base64,{2}" type="video/mp4" />
             </video>'''.format(960, 720, video_encoded.decode('ascii')))